In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from pymongo import MongoClient
from dotenv import load_dotenv

In [2]:
# Nama database dan collection
db_name = 'youtube_rewind_indonesia'
collection_label = 'labeling_2018'
collection_name = 'full_labeling_2018'

# Memuat value dari file .env
load_dotenv()

mongodb_url = os.getenv('URL_SANDY')
local_url = os.getenv('URL_LOCAL')

#### Mengambil data dari MongoDB ke dalam dataframe

In [3]:
# Membuat koneksi ke MongoDB
client = MongoClient(local_url)
db = client[db_name]  # Ganti dengan nama database Anda
labeling_2018_collection = db[collection_label]
full_labeling_2018_collection = db[collection_name]

# Mengambil data dari koleksi MongoDB
labeled_data = list(labeling_2018_collection.find())
unlabeled_data = list(full_labeling_2018_collection.find())

# Konversi data menjadi dataframe
df_labeled = pd.DataFrame(labeled_data)
df_unlabeled = pd.DataFrame(unlabeled_data)

# Memisahkan fitur dan label dari data yang berlabel
X_labeled = df_labeled['textOriginal']
y_labeled = df_labeled['label']

# Menggunakan TfidfVectorizer untuk mengubah teks menjadi fitur
vectorizer = TfidfVectorizer()
X_labeled_tfidf = vectorizer.fit_transform(X_labeled)

# Melatih model menggunakan Naive Bayes
model = MultinomialNB()
model.fit(X_labeled_tfidf, y_labeled)

# Memproses data yang tidak berlabel
X_unlabeled = df_unlabeled['textOriginal']
X_unlabeled_tfidf = vectorizer.transform(X_unlabeled)

# Memprediksi label untuk data yang tidak berlabel
y_unlabeled_pred = model.predict(X_unlabeled_tfidf)

# Menambahkan prediksi ke dalam dataframe data tidak berlabel
df_unlabeled['predicted_label'] = y_unlabeled_pred

# Mengupdate koleksi MongoDB dengan prediksi label
for index, row in df_unlabeled.iterrows():
    full_labeling_2018_collection.update_one(
        {"_id": row["_id"]},
        {"$set": {"predicted_label": row["predicted_label"]}}
    )

print("Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.")

Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.


In [4]:
df_positif = df_unlabeled[df_unlabeled['predicted_label'] == 'positif']
df_negatif = df_unlabeled[df_unlabeled['predicted_label'] == 'negatif']
df_netral = df_unlabeled[df_unlabeled['predicted_label'] == 'netral']

In [5]:
df_positif

,_id,textOriginal,predicted_label
2,6651962182f57e4e5420d12e,musik producernya emang eka kacaw asli kece pa...,positif
6,6651962182f57e4e5420d132,bagus sih lebih tarik sama,positif
7,6651962182f57e4e5420d133,yutub rewind keren dah inti,positif
8,6651962182f57e4e5420d134,congrats,positif
12,6651962182f57e4e5420d138,wah tambah keren bingit mantap sekali bydwy kece,positif
...,...,...,...
9990,6651962182f57e4e5420f832,gila sih mantep,positif
9992,6651962182f57e4e5420f834,the best,positif
9995,6651962182f57e4e5420f837,ngak sesuai ekspektasi sih,positif
9996,6651962182f57e4e5420f838,keren keren gw,positif


In [6]:
df_negatif

,_id,textOriginal,predicted_label
0,6651962182f57e4e5420d12c,chanel ada chef renata maen ome tv,negatif
3,6651962182f57e4e5420d12f,tu daylen teriak teriak kayak tai,negatif
19,6651962182f57e4e5420d13f,pesa nyambung jelas tidak kaya yt rewind pesa ...,negatif
22,6651962182f57e4e5420d142,yg ngedislike tau fans mro,negatif
23,6651962182f57e4e5420d143,emang bowo yutuber,negatif
...,...,...,...
9917,6651962182f57e4e5420f7e9,adegan mobil paling sampah bangsat,negatif
9933,6651962182f57e4e5420f7f9,yg dislike orang sirik biasa negara tetanga tuh,negatif
9957,6651962182f57e4e5420f811,daging anjing sayur kol mana jing,negatif
9970,6651962182f57e4e5420f81e,artis artis bulshit,negatif


In [7]:
df_netral

,_id,textOriginal,predicted_label
1,6651962182f57e4e5420d12d,bang erwin,netral
4,6651962182f57e4e5420d130,biasa sa cinta mana,netral
5,6651962182f57e4e5420d131,siapa kesini abis nonton rewind,netral
9,6651962182f57e4e5420d135,ada si bowongacawin aja,netral
10,6651962182f57e4e5420d136,ini kata sponsor baru liat gw,netral
...,...,...,...
9991,6651962182f57e4e5420f833,gin doang dih,netral
9993,6651962182f57e4e5420f835,bagus ga relatif hubung gue ga bikin begini ba...,netral
9994,6651962182f57e4e5420f836,jadi gin gaes knp banyak ytbr kaga ikt udh und...,netral
9997,6651962182f57e4e5420f839,nga scene kimihime renang padahal kan salah sa...,netral


#### Accuracy Test (soon)

In [8]:
# Jika data tidak berlabel memiliki label asli untuk keperluan evaluasi
if 'label' in df_unlabeled.columns:
    print("\nEvaluasi kinerja model:")
    print("Accuracy Score:", accuracy_score(df_unlabeled['label'], df_unlabeled['predicted_label']))
    print("Classification Report:\n", classification_report(df_unlabeled['label'], df_unlabeled['predicted_label']))
else:
    print("Data tidak berlabel tidak memiliki label asli untuk evaluasi.")

Data tidak berlabel tidak memiliki label asli untuk evaluasi.
